In [1]:
# Some basic imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import warnings
import ast
import re


In [2]:
# Load the general data:
success_movies_path = 'data/processed_data/success_movies.csv'
ethnicity_path = 'data/processed_data/diversity_final.csv'

success_movies_df = pd.read_csv(success_movies_path, sep=',')
ethnicity_df = pd.read_csv(ethnicity_path, sep=',')
success_movies_df.head()

test_df = ethnicity_df.merge(success_movies_df, how='inner', on='Wikipedia_movie_ID')
test_df['Ratings'].describe()

count    21481.000000
mean         6.208342
std          1.070229
min          1.200000
25%          5.600000
50%          6.300000
75%          6.900000
max          9.300000
Name: Ratings, dtype: float64

In [3]:
test_df['Movie_box_office_revenue'].describe()

count    2.085000e+03
mean     4.665568e+07
std      1.001648e+08
min      1.020000e+04
25%      2.000000e+06
50%      9.808124e+06
75%      3.811948e+07
max      9.260471e+08
Name: Movie_box_office_revenue, dtype: float64

## Regression


In [5]:
from sklearn.model_selection import train_test_split
# split the data into train and test sets using a 80-20 split
train, test = train_test_split(test_df, test_size=0.2, random_state=42)

# print number of samples in the training and test sets
print(f"The number of samples in the training set is: {train.shape[0]}")
print(f"The number of samples in the test set is: {test.shape[0]}")

The number of samples in the training set is: 17184
The number of samples in the test set is: 4297


In [ ]:
# split the data into features and target for the train and test datasets. 
# features contains all the columns minus the column price and label contains only the column price. 
